In [39]:
#Libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from string import printable
from collections import Counter
#from nltk.stem import WordNetLemmatizer

In [40]:
#load data
data = pd.read_csv('d1.csv',encoding='latin1')
data.head()

,ID,Tweet,Affect Dimension,intensity
0,2018-En-01052,we need to do something. something must be don...,anger,0.517
1,2018-En-03440,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,anger,0.726
2,2018-En-01489,Caleb had a nightmare about zombies.,anger,0.25
3,2018-En-00490,#CNN really needs to get out of the #Propagand...,anger,0.641
4,2018-En-02178,#dmme #kikme #sext #horny #ass #bbw #naught...,anger,0.313


In [41]:
label_encoder= preprocessing.LabelEncoder()
data['Affect Dimension']= label_encoder.fit_transform(data['Affect Dimension'])
data['Affect Dimension'].unique()

array([0, 1, 2, 3])

In [42]:
data.drop("intensity", axis=1, inplace=True)
data.drop("ID", axis=1, inplace=True)
data.head()

,Tweet,Affect Dimension
0,we need to do something. something must be don...,0
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0
2,Caleb had a nightmare about zombies.,0
3,#CNN really needs to get out of the #Propagand...,0
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7947 entries, 0 to 7946
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Tweet             7947 non-null   object
 1   Affect Dimension  7947 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 93.3+ KB


In [44]:
data.shape
data['Affect Dimension'].value_counts()

0    2035
1    2033
2    2003
3    1876
Name: Affect Dimension, dtype: int64

# Text cleaning

In [45]:
#cleaning text


lemmatizer = WordNetLemmatizer()



def CleanTxt(text):
    #text= str(text).lower() #lowercase
    text= re.sub(r'@[A-Za-z0-9]+','',text)
    text= re.sub(r'\_[A-Za-z0-9]+','',text) #remove mentions
    text= re.sub(r'#','',text) #remove hashtag
    text= re.sub(r'http:s?//\S+|www\.\S+','',text) #remove hyperlinks 
    text = re.sub("(.)\\1{2,}", "\\1", text)
    
    #text= text.translate(str.maketrans('','',string.punctuation)) #remove punctuations
   
    
    return text

data['Tweet_clean']= data['Tweet'].apply(CleanTxt)

data.head()

,Tweet,Affect Dimension,Tweet_clean
0,we need to do something. something must be don...,0,we need to do something. something must be don...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,We'd be fuming if the hijacked our Â£8m move ...
2,Caleb had a nightmare about zombies.,0,Caleb had a nightmare about zombies.
3,#CNN really needs to get out of the #Propagand...,0,CNN really needs to get out of the Propaganda ...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny ass bbw naughty puss...


In [46]:
#lemmatization


lemmatizer = WordNetLemmatizer()
wordnet_map={"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}


def lemmatize_words(txt):
    txt= str(txt).lower()
    pos_tagged_txt= nltk.pos_tag(txt.split())
    txt= " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_txt])
    return txt
data["Tweet_lema"]= data["Tweet_clean"].apply(lambda text:lemmatize_words(text))
data.head()

,Tweet,Affect Dimension,Tweet_clean,Tweet_lema
0,we need to do something. something must be don...,0,we need to do something. something must be don...,we need to do something. something must be don...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,We'd be fuming if the hijacked our Â£8m move ...,we'd be fume if the hijacked our â£8m move for...
2,Caleb had a nightmare about zombies.,0,Caleb had a nightmare about zombies.,caleb have a nightmare about zombies.
3,#CNN really needs to get out of the #Propagand...,0,CNN really needs to get out of the Propaganda ...,cnn really need to get out of the propaganda b...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny ass bbw naughty puss...,dmme kikme sext horny as bbw naughty pussy kik...


In [47]:
#dictionary of contractions
contractions_dict = { 
    
"&":"and",    
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"wot":"what",
"wtf":"what the fuck",
"wth":"what thw hell",    
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"pics":"pictures"    
}

In [48]:
#contractions from text
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)
data.drop("Tweet_clean", axis=1, inplace=True)
data["Tweet_lema"]= data["Tweet_lema"].apply(lambda text:expand_contractions(text))
data.head(148)

,Tweet,Affect Dimension,Tweet_lema
0,we need to do something. something must be don...,0,we need to do something. something must be don...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,we would be fume if the hijacked our â£8m move...
2,Caleb had a nightmare about zombies.,0,caleb have a nightmare about zombies.
3,#CNN really needs to get out of the #Propagand...,0,cnn really need to get out of the propaganda b...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny as bbw naughty pussy kik...
...,...,...,...
143,@0510Keisuke devour the unborn\nhuman rejectio...,0,devour the unborn\nhuman rejection\nfrom wrath...
144,Time for a #burst #burstcoin fork??,0,time for a burst burstcoin fork??
145,Is it supposed to be this arduous to interact ...,0,be it suppose to be this arduous to interact w...
146,@ehtesham_toi @RaviShastriOfc unfortunate deci...,0,"unfortunate decision, our player be more of su..."


In [49]:
#removal of punctuations
def remove_punc(text):
    text= text.translate(str.maketrans('','',string.punctuation)) #remove punctuations
    return text
data["Tweet_lema"]= data["Tweet_lema"].apply(lambda text:remove_punc(text))
data.head(148)

,Tweet,Affect Dimension,Tweet_lema
0,we need to do something. something must be don...,0,we need to do something something must be done...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,we would be fume if the hijacked our â£8m move...
2,Caleb had a nightmare about zombies.,0,caleb have a nightmare about zombies
3,#CNN really needs to get out of the #Propagand...,0,cnn really need to get out of the propaganda b...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny as bbw naughty pussy kik...
...,...,...,...
143,@0510Keisuke devour the unborn\nhuman rejectio...,0,devour the unbornnhuman rejectionnfrom wrath t...
144,Time for a #burst #burstcoin fork??,0,time for a burst burstcoin fork
145,Is it supposed to be this arduous to interact ...,0,be it suppose to be this arduous to interact w...
146,@ehtesham_toi @RaviShastriOfc unfortunate deci...,0,unfortunate decision our player be more of sup...


In [50]:
#to remove emojis from text
#from string import printable
#'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
data["Tweet_lema"]=data["Tweet_lema"].apply(lambda y: ''.join(filter(lambda x:  x in string.printable, y)))
data.head(20)


,Tweet,Affect Dimension,Tweet_lema
0,we need to do something. something must be don...,0,we need to do something something must be done...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,we would be fume if the hijacked our 8m move f...
2,Caleb had a nightmare about zombies.,0,caleb have a nightmare about zombies
3,#CNN really needs to get out of the #Propagand...,0,cnn really need to get out of the propaganda b...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny as bbw naughty pussy kik...
5,Don't fucking tag me in pictures as 'family fi...,0,do not fuck tag me in picture a family first w...
6,Coulda sworn it was Interview With A Vampire. ...,0,coulda swear it be interview with a vampire hm...
7,sick of this shit. #mad #angry. Rowan Atkinson...,0,sick of this shit mad angry rowan atkinson be ...
8,I despair at the mouth foaming hostility &amp;...,0,i despair at the mouth foam hostility andamp d...
9,@FangornForest1 @Brexit whose customers love i...,0,whose customer love it imagine be a hack these...


In [51]:
#removal of stop words
stop_words= set (stopwords.words("english"))
def lem(txt):
    
    #txt= word_tokenize(txt)
    txt= " ".join([word for word in (txt).split() if word not in stop_words])
    
    return txt

data['Tweet_lema']= data['Tweet_lema'].apply(lem)
data.head()

,Tweet,Affect Dimension,Tweet_lema
0,we need to do something. something must be don...,0,need something something must doneyour anxiety...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,would fume hijacked 8m move relegate full back
2,Caleb had a nightmare about zombies.,0,caleb nightmare zombies
3,#CNN really needs to get out of the #Propagand...,0,cnn really need get propaganda business 30 sec...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny bbw naughty pussy kik nu...


In [52]:
#from collections import Counter
cnt= Counter()
for text in data["Tweet_lema"].values:
    for word in text.split():
        cnt[word]+=1
        
        
cnt.most_common(10)

[('im', 740),
 ('get', 666),
 ('make', 509),
 ('like', 461),
 ('go', 435),
 ('andamp', 383),
 ('feel', 373),
 ('u', 357),
 ('day', 351),
 ('happy', 347)]

In [54]:
#removing the most common words
freq_words= set([w for (w, wc)in cnt.most_common(3)])

def remove_freq(txt):
    txt= " ".join([word for word in str(txt).split() if word not in freq_words])
    return txt
data["Tweet_lema"]= data["Tweet_lema"].apply(lambda text:remove_freq(text))
data.head(8)

,Tweet,Affect Dimension,Tweet_lema
0,we need to do something. something must be don...,0,need something something must doneyour anxiety...
1,@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineD...,0,would fume hijacked 8m move relegate full back
2,Caleb had a nightmare about zombies.,0,caleb nightmare zombies
3,#CNN really needs to get out of the #Propagand...,0,cnn really need propaganda business 30 second ...
4,#dmme #kikme #sext #horny #ass #bbw #naught...,0,dmme kikme sext horny bbw naughty pussy kik nu...
5,Don't fucking tag me in pictures as 'family fi...,0,fuck tag picture family first cut 5 year ago one
6,Coulda sworn it was Interview With A Vampire. ...,0,coulda swear interview vampire hmmandela effec...
7,sick of this shit. #mad #angry. Rowan Atkinson...,0,sick shit mad angry rowan atkinson dead bloody...


In [56]:
data.to_csv('./data_clean.csv', index=False)